In [1]:
# QUAN TRỌNG: Nếu gặp lỗi C compiler, hãy chạy lệnh sau trong môi trường của bạn và khởi động lại kernel:
# !apt-get update && apt-get install -y build-essential

import os
import json
import glob
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from datasets import Dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model

# ==============================================================================
# 1. CẤU HÌNH & HYPERPARAMETERS
# ==============================================================================
class Config:
    MODEL_ID = "infosys/NT-Java-1.1B"
    DATA_DIR = "/code-smell-detection/data/finetune_data_augmented"
    OUTPUT_DIR = "/code-smell-detection/code-smell-finetuned-model-masked-loss"
    RANDOM_SEED = 42
    TEST_SIZE = 0.10
    DEV_SIZE_FROM_TEST = 0.50
    LORA_R = 4
    LORA_ALPHA = 4
    LORA_DROPOUT = 0.1
    LORA_TARGET_MODULES = ["c_attn", "c_proj", "c_fc"]
    NUM_TRAIN_EPOCHS = 50 
    EARLY_STOPPING_PATIENCE = 5
    PER_DEVICE_TRAIN_BATCH_SIZE = 3
    PER_DEVICE_EVAL_BATCH_SIZE = 3
    GRADIENT_ACCUMULATION_STEPS = 8
    LEARNING_RATE = 1e-3
    WEIGHT_DECAY = 0.1
    WARMUP_RATIO = 0
    FP16 = True

config = Config()

# ==============================================================================
# 2. TẢI VÀ TIỀN XỬ LÝ DỮ LIỆU
# ==============================================================================
def load_data_from_json_files(data_dir):
    """
    Tải dữ liệu từ các file .json.
    Label được xác định là chuỗi văn bản chính xác nằm sau <fim_middle>.
    """
    json_files = glob.glob(os.path.join(data_dir, "*.json"))
    data = []
    
    for file_path in json_files:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = json.load(f)
            text = content.get("text", "")
            
            try:
                # Trích xuất phần label thô
                label_part_raw = text.split("<fim_middle>")[1]
                # Thêm dữ liệu vào danh sách.
                data.append({"text": text, "label": label_part_raw})

            except IndexError:
                print(f"Cảnh báo: Định dạng FIM không đúng trong file {file_path}. Bỏ qua.")
                continue
                
    return data

def create_datasets(data, config):
    train_data, temp_data = train_test_split(
        data, test_size=config.TEST_SIZE, random_state=config.RANDOM_SEED, stratify=[d['label'] for d in data]
    )
    test_data, dev_data = train_test_split(
        temp_data, test_size=config.DEV_SIZE_FROM_TEST, random_state=config.RANDOM_SEED, stratify=[d['label'] for d in temp_data]
    )
    return DatasetDict({
        "train": Dataset.from_list(train_data),
        "validation": Dataset.from_list(dev_data),
        "test": Dataset.from_list(test_data)
    })

print("Bắt đầu tải và xử lý dữ liệu...")
raw_data = load_data_from_json_files(config.DATA_DIR)
if not raw_data:
    raise ValueError("Không có dữ liệu nào được tải. Vui lòng kiểm tra lại đường dẫn DATA_DIR.")
datasets = create_datasets(raw_data, config)
print("Thông tin bộ dữ liệu:")
print(datasets)

# ==============================================================================
# 3. THIẾT LẬP TOKENIZER VÀ MODEL
# ==============================================================================
print(f"Tải tokenizer và model '{config.MODEL_ID}' (không quantization)...")
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    config.MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True, remove_columns=["text", "label"])
print("Tokenize dữ liệu hoàn tất.")

# ==============================================================================
# 4. TÍNH TOÁN TRỌNG SỐ CLASS (CLASS WEIGHTS)
# ==============================================================================
train_labels = datasets['train']['label']
class_names = sorted(list(set(train_labels)))
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.array(class_names),
    y=train_labels
)
class_weights_map = {name: weight for name, weight in zip(class_names, class_weights_array)}
class_weights_tensor = torch.tensor(list(class_weights_map.values()), dtype=torch.float32).to("cuda")
label2id = {name: i for i, name in enumerate(class_names)}

print(f"Các lớp được tìm thấy ({len(class_names)} lớp): {class_names}")
print(f"Trọng số class được tính toán: {class_weights_map}")

tokenized_train_with_ids = tokenized_datasets['train'].add_column("label_id", [label2id[l] for l in datasets['train']['label']])
tokenized_validation_with_ids = tokenized_datasets['validation'].add_column("label_id", [label2id[l] for l in datasets['validation']['label']])

# ==============================================================================
# 5. THIẾT LẬP DATA COLLATOR VÀ TRAINER TÙY CHỈNH
# ==============================================================================

class DataCollatorForFIMTask:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.fim_middle_token_id = tokenizer.convert_tokens_to_ids("<fim_middle>")

    def __call__(self, features):
        label_ids = [feature.pop("label_id") for feature in features]
        batch = self.tokenizer.pad(features, return_tensors="pt", padding=True)
        batch["label_id"] = torch.tensor(label_ids, dtype=torch.long)
        labels = batch["input_ids"].clone()
        
        for i in range(labels.shape[0]):
            input_ids_list = batch["input_ids"][i].tolist()
            try:
                middle_token_index = input_ids_list.index(self.fim_middle_token_id)
                labels[i, : middle_token_index + 1] = -100
            except ValueError:
                labels[i, :] = -100
        
        labels[labels == self.tokenizer.pad_token_id] = -100
        batch["labels"] = labels
        return batch

class WeightedLossTrainer(Trainer):
    def __init__(self, *args, class_weights, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        label_ids = inputs.pop("label_id")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        labels = inputs.get("labels")
        loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
        loss = loss_fct(logits.view(-1, self.model.config.vocab_size), labels.view(-1))
        loss_per_sequence = loss.view(labels.shape[0], -1).sum(dim=1)
        num_active_tokens = (labels.view(labels.shape[0], -1) != -100).sum(dim=1)
        mean_loss_per_sequence = loss_per_sequence / (num_active_tokens + 1e-8)
        weights_for_batch = self.class_weights[label_ids].to(mean_loss_per_sequence.device)
        weighted_loss_per_sequence = mean_loss_per_sequence * weights_for_batch
        final_loss = weighted_loss_per_sequence.mean()
        return (final_loss, outputs) if return_outputs else final_loss

data_collator = DataCollatorForFIMTask(tokenizer=tokenizer)

# ==============================================================================
# 6. THIẾT LẬP VÀ CHẠY HUẤN LUYỆN
# ==============================================================================
peft_config = LoraConfig(
    r=config.LORA_R,
    lora_alpha=config.LORA_ALPHA,
    lora_dropout=config.LORA_DROPOUT,
    target_modules=config.LORA_TARGET_MODULES,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)
print("Cấu hình LoRA hoàn tất. Các tham số có thể huấn luyện:")
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir=config.OUTPUT_DIR,
    num_train_epochs=config.NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=config.PER_DEVICE_TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=config.PER_DEVICE_EVAL_BATCH_SIZE,
    gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
    learning_rate=config.LEARNING_RATE,
    weight_decay=config.WEIGHT_DECAY,
    warmup_ratio=config.WARMUP_RATIO,
    fp16=config.FP16,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    remove_unused_columns=False,
)

trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_with_ids,
    eval_dataset=tokenized_validation_with_ids,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=config.EARLY_STOPPING_PATIENCE)],
    class_weights=class_weights_tensor 
)

print("\nBắt đầu quá trình finetune...")
trainer.train()
print("Quá trình finetune đã hoàn tất!")

final_model_path = os.path.join(config.OUTPUT_DIR, "final_best_model")
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)
print(f"Model tốt nhất đã được lưu tại: {final_model_path}")

Bắt đầu tải và xử lý dữ liệu...
Thông tin bộ dữ liệu:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4025
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 224
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 224
    })
})
Tải tokenizer và model 'infosys/NT-Java-1.1B' (không quantization)...


Map:   0%|          | 0/4025 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/224 [00:00<?, ? examples/s]

Map:   0%|          | 0/224 [00:00<?, ? examples/s]

Tokenize dữ liệu hoàn tất.
Các lớp được tìm thấy (10 lớp): ['// class below has no smell\n', '// class below is blob\n', '// class below is blob and data class\n', '// class below is data class\n', '// class below is data class and blob\n', '// function below has no smell\n', '// function below is feature envy\n', '// function below is feature envy and long method\n', '// function below is long method\n', '// function below is long method and feature envy\n']
Trọng số class được tính toán: {'// class below has no smell\n': 0.2810754189944134, '// class below is blob\n': 1.8896713615023475, '// class below is blob and data class\n': 9.147727272727273, '// class below is data class\n': 1.7055084745762712, '// class below is data class and blob\n': 9.147727272727273, '// function below has no smell\n': 0.23079128440366972, '// function below is feature envy\n': 10.320512820512821, '// function below is feature envy and long method\n': 7.453703703703703, '// function below is long method\n

/tmp/ipykernel_1028406/2125370872.py:164: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Cấu hình LoRA hoàn tất. Các tham số có thể huấn luyện:
trainable params: 2,777,088 || all params: 1,139,984,384 || trainable%: 0.2436

Bắt đầu quá trình finetune...


Epoch,Training Loss,Validation Loss
1,0.000000,0.000000
2,0.000000,0.000000
3,0.000000,0.000000
4,0.000000,0.000000
5,0.000000,0.000000
6,0.000000,0.000000
7,0.000000,0.000000
8,0.000000,0.000000
9,0.000000,0.000000
10,0.000000,0.000000


Quá trình finetune đã hoàn tất!
Model tốt nhất đã được lưu tại: /code-smell-detection/code-smell-finetuned-model-masked-loss/final_best_model


# test

In [2]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# ==============================================================================
# 1. CẤU HÌNH
# ==============================================================================
# ID của mô hình gốc trên Hugging Face
base_model_id = "infosys/NT-Java-1.1B"

# Đường dẫn đến adapter LoRA đã được finetune và lưu lại
adapter_path = "/code-smell-detection/code-smell-finetuned-model-masked-loss/final_best_model"

# ==============================================================================
# 2. TẢI MODEL VÀ TOKENIZER
# ==============================================================================
print("Bắt đầu tải mô hình và tokenizer...")

# Thiết lập thiết bị (sử dụng GPU nếu có)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Tải mô hình gốc với định dạng bfloat16 để tối ưu bộ nhớ
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16,
)

# Tải tokenizer từ thư mục adapter để đảm bảo tính nhất quán
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

# Tải và hợp nhất adapter LoRA vào mô hình gốc
# Thao tác này sẽ tạo ra mô hình đã được finetune hoàn chỉnh
model = PeftModel.from_pretrained(base_model, adapter_path)

# Chuyển mô hình sang thiết bị và đặt ở chế độ đánh giá (evaluation)
model = model.to(device)
model.eval()

print(f"Mô hình đã được tải thành công lên thiết bị: {device}")

# ==============================================================================
# 3. CHUẨN BỊ DỮ LIỆU TEST
# ==============================================================================

# Ví dụ 1: Code "sạch", không có smell
# Mong đợi: model sẽ sinh ra "// class below has no smell"
input_text = "<fim_prefix>public class PalindromeChecker {\n        public static boolean isPalindrome(String str) {\n          <fim_suffix>return true;\n      }\n<fim_middle>"

code_prompt = "<fim_prefix>/*\n* Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.\n*\n* Licensed under the Apache License, Version 2.0 (the \"License\"). You may not use this file\n* except in compliance with the License. A copy of the License is located at\n*\n* http://aws.amazon.com/apache2.0/\n*\n* or in the \"license\" file accompanying this file. This file is distributed on an \"AS IS\" BASIS,\n* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for\n* the specific language governing permissions and limitations under the License.\n*/\n\npackage com.amazon.ask.dispatcher.request.handler.impl;\n\nimport com.amazon.ask.dispatcher.request.handler.HandlerInput;\nimport com.amazon.ask.dispatcher.request.handler.RequestHandler;\nimport com.amazon.ask.model.interfaces.audioplayer.PlaybackNearlyFinishedRequest;\nimport com.amazon.ask.model.Response;\n\nimport java.util.Optional;\n\n/**\n * Request handler for PlaybackNearlyFinishedRequest requests.\n */\n<fim_suffix>public interface PlaybackNearlyFinishedRequestHandler extends RequestHandler {\n\n    /**\n     * Returns true if the handler can dispatch the current request\n     *\n     * @param input input to the request handler\n     * @param playbackNearlyFinishedRequest PlaybackNearlyFinishedRequest request\n     * @return true if the handler is capable of handling the current request and/or state\n     */\n    boolean canHandle(HandlerInput input, PlaybackNearlyFinishedRequest playbackNearlyFinishedRequest);\n\n    /**\n     * Handles the request.\n     *\n     * @param input input to the request handler\n     * @param playbackNearlyFinishedRequest PlaybackNearlyFinishedRequest request\n     * @return output from the handler.\n     */\n    Optional<Response> handle(HandlerInput input, PlaybackNearlyFinishedRequest playbackNearlyFinishedRequest);\n\n    @Override\n    default boolean canHandle(HandlerInput handlerInput) {\n        if (handlerInput.getRequest() instanceof PlaybackNearlyFinishedRequest) {\n            return canHandle(handlerInput, (PlaybackNearlyFinishedRequest)handlerInput.getRequest());\n        }\n        return false;\n    }\n\n    @Override\n    default Optional<Response> handle(HandlerInput handlerInput) {\n        return handle(handlerInput, (PlaybackNearlyFinishedRequest)handlerInput.getRequest());\n    }\n\n}<fim_middle>"

# // class below has no smell\n
test_prompts = {
    "Test Case 1: Code": code_prompt,
}

# ==============================================================================
# 4. CHẠY INFERENCE
# ==============================================================================

for test_name, prompt in test_prompts.items():
    print("\n" + "="*50)
    print(f"Đang chạy: {test_name}")
    print("="*50)
    print("--- INPUT PROMPT ---")
    print(prompt)
    
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Sinh ra output từ model
    # Chúng ta chỉ cần sinh ra một đoạn text ngắn (label)
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=50, 
            eos_token_id=tokenizer.eos_token_id
        )

    # Decode và hiển thị kết quả
    # Kết quả sẽ bao gồm cả prompt đầu vào, chúng ta sẽ in toàn bộ để xem
    result = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    print("\n--- MODEL OUTPUT ---")
    print(result)
    print("="*50)

print("\nHoàn tất kiểm tra inference.")

Bắt đầu tải mô hình và tokenizer...


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Mô hình đã được tải thành công lên thiết bị: cuda

Đang chạy: Test Case 1: Code
--- INPUT PROMPT ---
<fim_prefix>/*
* Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
*
* Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file
* except in compliance with the License. A copy of the License is located at
*
* http://aws.amazon.com/apache2.0/
*
* or in the "license" file accompanying this file. This file is distributed on an "AS IS" BASIS,
* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for
* the specific language governing permissions and limitations under the License.
*/

package com.amazon.ask.dispatcher.request.handler.impl;

import com.amazon.ask.dispatcher.request.handler.HandlerInput;
import com.amazon.ask.dispatcher.request.handler.RequestHandler;
import com.amazon.ask.model.interfaces.audioplayer.PlaybackNearlyFinishedRequest;
import com.amazon.ask.model.Response;

import java.util.